First of all, we need to get rid of all NA's. 
To this end, we will use the pandas' ffill method: propagate last valid observation forward to next valid backfill
It is important to note that **propagate observations** are only needed **inside the time series for each individual customer**.
Also, if the time series **started with NA's**, it is more logical to fill in these gaps not with the first available observation, but **with the mean** (for numerical features) and **with the mode** (for categorical features) over the whole time series for customer


In [1]:
import pandas as pd
import pickle

In [8]:
%%time

data = pd.read_pickle('train_data.pkl')

Wall time: 7.94 s


In [5]:
cat_feats = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
feats = [col for col in data if col not in ["customer_ID",'S_2']]
num_feats = [col for col in feats if col not in cat_feats]
# ids = data["customer_ID"]
# un_ids = pd.Series(ids.unique())

Let's transform datatypes in order to reduce size

In [61]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5531451 entries, 0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a to fffff1d38b785cef84adeace64f8f83db3a0c31e8d92eaba8b115f71cab04681
Columns: 190 entries, customer_ID to D_145
dtypes: float64(185), int64(1), object(4)
memory usage: 7.9+ GB


In [71]:
data.dtypes.unique()

array([dtype('O'), dtype('float64'), dtype('int64')], dtype=object)

In [74]:
data[data.dtypes[data.dtypes == "float64"].index] = data[data.dtypes[data.dtypes == "float64"].index].astype("float32",copy=False)

In [ ]:
data.dtypes[data.dtypes == "int64"].index

In [84]:
data["B_31"].unique()

array([1, 0], dtype=int64)

In [85]:
data[data.dtypes[data.dtypes == "int64"].index] = data[data.dtypes[data.dtypes == "int64"].index].astype("int8",copy=False)

In [95]:
data[[i for i in cat_feats if i not in data.dtypes[data.dtypes == "object"].index]].nunique()

B_30     3
B_38     7
D_114    2
D_116    2
D_117    7
D_120    2
D_126    3
D_66     2
D_68     7
dtype: int64

In [103]:
data[[i for i in cat_feats if i not in data.dtypes[data.dtypes == "object"].index]] = data[[i for i in cat_feats if i not in data.dtypes[data.dtypes == "object"].index]].astype("float16")

In [107]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5531451 entries, 0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a to fffff1d38b785cef84adeace64f8f83db3a0c31e8d92eaba8b115f71cab04681
Columns: 190 entries, customer_ID to D_145
dtypes: float16(9), float32(176), int8(1), object(4)
memory usage: 4.0+ GB


 Such a progress! (7.9+ GB -> 4.0+ GB) :)

In [ ]:
%%time

gr_data = data.groupby("customer_ID")

In [ ]:
%%time

data.gr_data.fillna(method="ffill", inplace=True)

In [ ]:
#Не работает!

data[num_feats] = gr_data[num_feats].transform(lambda x: x.fillna(x.mean()))

In [24]:
%%time

avgs = []
i=1
for name, group in gr_data[num_feats]:
    avgs.append(
        [name, *group.mean()]
    )
    print(i, "from", 458913)
    i +=1
avgs = pd.DataFrame(avgs, columns=["customer_ID", *num_feats])

Wall time: 14.1 s


In [ ]:
data.set_index(data["customer_ID"])
avgs.set_index(avgs["customer_ID"])

In [3]:
#Туго, но сработало))

data[num_feats] = data[num_feats].fillna(avgs)

In [138]:
%%time
gr_cats = data[cat_feats].groupby("customer_ID") 

Wall time: 116 ms


In [ ]:
from tqdm import tqdm
tqdm.pandas()

data[cat_feats] = gr_cats.progress_transform(lambda x: x.fillna(x.mode()))

In [169]:
#data export

data.to_pickle("data_compressed.zip")